In [7]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-success">
    <h1>Problema 1: Repostatge de vehicles (No òptim) (3p)</h1>
    <p>
       Hem de fer un trajecte des d’un punt d’origen $S$ fins a un destí $D$. El dipòsit del cotxe permet recórrer un màxim de $K$ quilòmetres. A més, al llarg del trajecte, trobem un conjunt de benzineres $B_1,\dots,B_n$. Cada benziera té la benzina a un preu diferent que podem notar com $p_{B_i},\ \ \forall i=1,\dots,n$. Implementeu l'algorisme greedy descrit a continuació per anar des de $S$ fins a $D$.<br><br>
       Implementeu un algorisme greedy que, donat $K$, una llista de distàncies entre l'orígen i les benzineres on l'últim element és el destí, i una llista  de preus de benzina, retorni el nombre de cops que haurem de parar a repostar i el preu total del trajecte.<br><br>
       Per exemple, si tenim un cotxe que pot fer 10km sense repostar, el destí està a 30km i tenim benzineres als punts: 8, 14, 16, 18, 23, 27 amb preus 0.9, 1.2, 0.6, 1.2, 2.1, 1.7 respectivament, podem executar:<br><br><center><b>refill_prices(10, [8, 14, 16, 18, 23, 27, 30], [0.9, 1.2, 0.6, 1.2, 2.1, 1.7])</center><br> i ens haurà de retornar quatre valors:
        <ul>
            <li> <b>True/False</b> depenent de si existeix, o no, una solució al problema.
            <li> <b>Nombre de benzineres on hem de fer parada.</b> En cas que no existeixi solució, retornarà el nombre de benzineres que hem visitat abans d'exhaurir el carburant.
            <li> <b>Llista dels quilòmetres que formen part de la solució.</b>
            <li> <b>Cost total del trajecte. </b>
        </ul></b><b></b>
    </p>    
    Seguiu el següent esquema a l'hora de programar la vostra solució:<br>
    <ol>
        <li> Ens movem fins la benzinera de més baix cost a la que podem arribar.
        <li> Omplim el dipòsit al màxim.
        <li> Busquem la propera benzinera a la que podem arribar amb el cost més baix.
        <li> Tornem al punt 1.
    </ol>
    
</div>

In [8]:
def refill_prices(K, stations, prices):
    """
    Repostatge no òptim de vehicles amb costos.
    
    Params
    ======
    :K: dipòsit del vehicle
    :stations: llista de benzineres. L'últim element és el destí.
    :prices: Llista de preus. Té un element menys que 'stations'.
    
    Returns
    =======
    :exists: Booleà True/False depenent de si existeix o no solució al problema.
    :num_stops: Número de benzineres a les que hem de parar.
    :stops: Quilòmetres de les benzineres on fem parada.
    :value: Cost del trajecte.
    """
    
    exists = False    
    stops = []
    num_stops = 0
    value = 0.0
    km = K
    cheap = 0, float('inf')
    paradas = 0
    i = 0

    # Bucle per recorrer totes les estacions
    while i <= len(stations) - 1:

        # Si podem arribar a la estacio amb la gasolina actual
        if stations[i] <= km:
            # I es mes barata que la posible anterior
            if prices[i] < cheap[1]:
                paradas = i
                cheap = stations[i], prices[i]
        # Si no arribem a la seguent, anem a la que hem guardat com a mes barata (cheap)
        else: 
            stops.append(cheap[0])
            num_stops += 1
            # Actualitzem els km als que podem arribar amb la gasolina actual
            km = stops[-1] + K
            
            # Si estem en el primer cas (len(stops) == 1) hem de repostar la quantitat de km per arribar a la primera parada
            # en cas contrari, hem de repostar la diferencia de km entre la parada a la que volem arribar i la que ens trobem
            value += cheap[1] * (stops[-1] if len(stops) == 1 else stops[-1] - stops[-2])

            cheap = 0, float('inf')

            # Actualitzem i per a tornar a la parada en la que ens trobem
            i = paradas

            # Si podem arribar a la ultima parada, parem
            if km >= stations[-1]:
                value = round(value, 2)
                exists = True
                break
        i += 1

    return exists, num_stops, stops, value

In [ ]:
print(refill_prices(10, [10, 11, 13, 14], [0.5, 0.4, 0.3]))

In [ ]:
print(refill_prices(10, [8, 14, 16, 18, 23, 27, 30], [0.9, 1.2, 0.6, 1.2, 2.1, 1.7]))

<div class="alert alert-success">
    <h1>Problema 2: Repostatge de vehicles (Òptim) (2p)</h1>
    <p>
       Modifiqueu l'algorisme anterior per a que la solució sigui òptima. <br>
       En aquest cas, només haureu d'omplir el dipòsit amb els litres necessaris per a arribar a la benzinera més propera amb un cost inferior a la que estem actualment. <br>
       Comproveu que la solució que obteniu amb aquest algorisme és inferior a la solució del Problema 1.
    </p>    
   
</div>

Exemple:<br>
       Suposem que tenim benzineres als punts 10, 11, 13, el destí està al punt 14 i els preus són 0.5, 0.4, 0.3 respectivament. La capacitat del cotxe és de 10L.<br>
       <ol>
           <li> Comprovem quines benzineres són assolibles: Només podem arribar a la benzinera del km.10 que té cost 0.5€/L.
           <li> Abans de decidir quants litres omplim, mirem a quines benzineres podríem arribar. Podríem arribar al km.11, al km.13 i al km.14 (destinació).
           <li> Com que la benzinera del km.11 val 0.4€/L i 0.4<0.5, només omplim amb els litres necessaris per arribar a aquesta benzinera (1L).
           <li> Ens movem a aquesta benzinera i repetim el procés.                                                    
       </ol>

In [11]:
def refill_prices_optim(K, stations, prices):
    
    # Quan tenim el tanc ple i per tant hem de veure quina és la gasolinera més barata a la que podem arribar
    def gasolinera_mes_barata(stations, prices, K, km_act):
        # Calcular i en relacio al la posicio actual
        i = 0 if km_act == 0 else stations.index(km_act) + 1
        
        trobat = False
        cheap = 0, float('inf')

        # Recorrem stations mentre poguem arribar a l'estacio
        while i <= len(stations) - 1 and (stations[i] - km_act) <= K:
            trobat = True

            # Si trobem un preu mes barat que l'anterior, actualtizem
            if prices[i] < cheap[1]:
                cheap = stations[i], prices[i]
            i += 1

        # Si no hem pogut arribar a cap estacio retornem -1, en altre cas retornem la parada mes barata a la que podem arribar
        return cheap if trobat else (-1,-1)
    
    # Quan no tenim el tanc ple i per tant hem de veure si tenim una gasolinera mes barata que la actual
    def gasolinera_barata(stations, prices, K, actual_price, km_act):
        # Calcular "i" en relacio al la posicio actual
        i = stations.index(km_act) + 1
        trobat = False

        # Si ens trobem a la penultima estacio i podem arribar al final retornem -1, si no podem arribar -3
        if i == len(stations) - 1:
            return -3 if stations[i - 1] + K < stations[i] else -1
            
        # Recorrem stations mentre poguem arribar a l'estacio
        while i <= len(stations) - 1 and (stations[i] - km_act) <= K:
            trobat = True

            # Si trobem un preu mes barat que l'actual, retonem l'index
            if prices[i] < actual_price:
                return i
            i += 1
        
        # Si hem sortit del bucle i no hem trobat cap parada mes barata que l'actual, pero, es posible arribar a alguna, retornem -2
        # Si no hem trobat cap parada a la que podem arribar, retornem -3
        return -2 if trobat else -3

    """
    Repostatge òptim de vehicles amb costos.
    
    Params
    ======
    :K: dipòsit del vehicle
    :stations: llista de benzineres. L'últim element és el destí.
    :prices: Llista de preus. Té un element menys que 'stations'.
    
    Returns
    =======
    :exists: Booleà True/False depenent de si existeix o no solució al problema.
    :num_stops: Número de benzineres a les que hem de parar.
    :stops: Quilòmetres de les benzineres on fem parada.
    :value: Cost del trajecte.
    """
    
    exists = False  
    value = 0.0
    i = 0

    # Cas per arribar a la primera parada
    cheap = gasolinera_mes_barata(stations, prices, K, 0)
    
    # Si no arribem a cap parada retornem False
    if cheap == (-1, -1):
        exists = False
        return exists, 0, [], value
    
    stops = [cheap[0]]
    num_stops = 1
    deposit = K - cheap[0]
    km_act = cheap[0]
    cheap_ant = cheap

    # Bucle per recorrer totes les estacions
    while i <= len(stations) - 1:

        # Calculem el index de la gasolinera propera mes barata que la actual
        index = gasolinera_barata(stations, prices, K, cheap[1], km_act)

        # Si no pots arribar a cap gasolinera
        if index == -3:
            exists = False
            break

        # Si la gasolinera en la que ens trobem és la més barata dins les que tenim al nostre abast
        # Omplim el deposit
        elif index == -2:
            value += cheap_ant[1] * (K - deposit)
            deposit = K
            cheap = gasolinera_mes_barata(stations, prices, K, km_act)

        # Si des de la teva gasolinera pots arribar al desti
        elif index == -1:
            km_trip = stations[-1] - km_act
            value += cheap_ant[1] * (km_trip - deposit)
            deposit += km_trip
            exists = True
            break
        
        # Si es troba una gasolinera propera mes barata que la actual
        else:
            cheap = stations[index], prices[index]

        km_trip = cheap[0] - km_act

        # Si no arribem amb el deposit actual a la parada desitjada, hem de repostar els litres necessaris
        if(deposit < km_trip):
            value += cheap_ant[1] * (km_trip - deposit)
            deposit = km_trip

        km_act = cheap[0]
        deposit -= km_trip
        stops.append(cheap[0])
        num_stops += 1
        cheap_ant = cheap

        i += 1

    value = round(value,2)
    
    return exists, num_stops, stops, value


In [ ]:
print(refill_prices_optim(10, [10, 11, 13, 14], [0.5, 0.4, 0.3]))

In [ ]:
print(refill_prices_optim(10, [8, 14, 16, 18, 23, 27, 30], [0.9, 1.2, 0.6, 1.2, 2.1, 1.7]))

In [ ]:
print(refill_prices_optim(5, [5, 10, 14, 18, 23, 24, 30], [0.9, 1.2, 0.6, 1.2, 2.1, 1.7]))

In [ ]:
print(refill_prices_optim(5, [8, 10, 14, 18, 23, 24, 30], [0.9, 1.2, 0.6, 1.2, 2.1, 1.7]))

In [ ]:
print(refill_prices_optim(10, [8, 19, 16, 18, 23, 27, 30], [0.9, 1.2, 0.6, 1.2, 2.1, 1.7]))

<div class="alert alert-success">
    <h1>Problema 3: Un alfabet estrany (5p)</h1>
    <p>
        Volem enviar missatges utilitzant el mínim d'espai possible utilitzant una codificació de dos caràcters '.' i '-'. Per fer-ho hem de traduïr cada lletra de l'alfabet català a aquesta representació. <br>
        Per exemple, podem assignar que la lletra A correspon a l'string '.', la lletra B a l'string '.-' i la lletra C a l'string '--'. Així, la paraula ABC ens quedaria codificada com '..---'.<br><br>
        Per a que la codificació sigui bona i reversible, és a dir, que siguem capaços de desxifrar-la, és important que els strings de codificació que triem no portin a errors.<br>
        Per exemple, si codifiquem A='.', B='-.', C='.-' i tenim l'string '.-..-', aquest pot representar tant la paraula ABC com la paraula CAC i, per tant, no és una bona codificació.
        Seguiu els següents passos per a implementar la solució:
    </p>  
    <h2>3.1 Trobar la codificació</h2>
    <ol>
        <li>Implementeu una funció <code>compute_frequency</code> que calculi la freqüència de cada lletra, és a dir, el nombre de cops que apareix a dins l'string.   
        <li>Ara crearem un arbre binari per emmagatzemar els nodes. Utilitza la classe <code>Node</code> seguint les indicacions:
        <ol>
            <li>Inicialitza una llista <code>nodes_list</code> on cada lletra sigui un node i tingui com a valor, el càlcul de freqüència del primer apartat.
            <li>Mentre la llista tingui dos nodes com a mínim, extreu de la llista els dos nodes amb la frequencia més petita i guarda'ls a l'arbre assignant-els-hi el mateix pare. 
            <li>Assigna el caràcter '.' al node de la dreta i el caràcter '-' al node de l'esquerra.
            <li>Guarda el node pare a <code>nodes_list</code> assignant-li com a valor de freqüència la suma dels seus dos fills.
            <li>Torna al punt B.
        </ol>
        <img src="img/graph.png" width='25%'></img><br>
        <li>Assigna un codi a cada fulla de l'arbre de la forma següent: sempre que descendeixis a un node de la dreta utilitza el caràcter '.' i al descendir al node de l'esquerra utilitza el caràcter '-'.
        <li>Retorna aquesta assignació com a diccionari.
    </ol>
    <h2>3.2 Codificar i decodificar</h2>
    <ol>
        <li>Crea una funció <code>encode</code> que utilitzi el diccionari retornat anteriorment per a codificar un text d'entrada
        <li>Crea una funció <code>decode</code> que utilitzi el diccionari retornat anteriorment per a decodificar un text d'entrada
        <li>Executa la funció <code>alphabet</code> i comprova que el resultat sigui l'esperat.
    </ol>
    
</div>

Exemple d'execució:

- Text a codificar: ABABAC
- Diccionari de freqüències: {'A': 3, 'B': 2, 'C': 1}
- Diccionari de conversió: {'C': '--', 'B': '-.', 'A': '.'}
- Codificat: .-..-..--
- Decodificat: ABABAC

In [17]:
from collections import defaultdict

def compute_frequency(text):
    """
    Params
    ======
    :text: El text que volem codificar
    
    Returns
    =======
    :dct: Un diccionari amb el nombre de cops que apareix cada simbol en el text d'entrada. Per exemple: {'A': 3, 'B': 2, 'C': 1}
    """
    dct = {}

    for letter in text:
        if letter not in dct:
            dct[letter] = 1
        else:
            dct[letter] += 1
    
    
    return dct

In [18]:
class Node:
    """
    Aquesta classe emmagatzema la informació dels nodes de l'arbre binari.
    """
    def __init__(self, node, value, left=None, right=None):
        self.node = node    # String que representa el node
        self.value = value  # Valor de freqüència
        self.left = left    # Node de l'esquerra
        self.right = right  # Node de la dreta
        self.code = ''      # Codificació del node

    def set_code(self, code):
        self.code = code

    def __repr__(self):
        return f"Node({self.node},{self.value},{self.code})"

In [19]:
def assign_codes(text, counts): 

    # Funcio recursiva per crear un diccionari a partir de l'arbre
    def create_dict(node_list, node, n, code_pare):
        # Si es node esquerre li posem el code del pare mes un guio, en cas contrari un punt
        if n == 0:
            node.set_code(code_pare + "-")
        else:
            node.set_code(code_pare + ".")
        # Si te fills cridem a la recursiva 
        if node.left != None:
            create_dict(node_list, node.left, 0, node.code)
        if node.right != None:
            create_dict(node_list, node.right, 1, node.code)
        
    """
    Aquesta funció construeix el diccionari de conversió de lletres a símbols '.' i '-'.
    
    Params
    ======
    :text: El text que volem convertir
    :counts: El diccionari de freqüències que ens retorna la funció compute_frequency
    
    Returns
    =======
    :codes: El diccionari de conversió. Per exemple: {'C': '--', 'B': '-.', 'A': '.'}
    """
    codes = {}
    node_list = []

    # Ordenem counts, respecte la frequencia de cada lletra
    sorted_counts = sorted(counts.items(), key=lambda item: item[1])

    # Si hi han 1 o 0, retornem la llista amb el cas especial
    if len(sorted_counts) < 1:
        if len(sorted_counts) == 1: 
            codes[counts.values] = "."
        return codes
    
    # Agafem les dues primeres fulles de l'arbre (les dues lletres amb menor frequencia) i creem el primer pare
    # Ho fem degut a que es un cas especial perque sera l'unic pare amb dos fills lletra (els altres tindran un fill lletra i un fill pare)
    letter1 = sorted_counts[0]
    letter2 = sorted_counts[1]
    valor_ant = letter1[1] + letter2[1]

    esq = Node(letter1[0], letter1[1])
    dr = Node(letter2[0], letter2[1])
    pare = Node(valor_ant, -1, esq, dr)
    
    pare_ant = pare
    
    sorted_counts.pop(0)
    sorted_counts.pop(0)

    node_list.append(esq)
    node_list.append(dr)
    node_list.append(pare)

    # Anem afegint la resta de fulles creant els seus respectius pares amb l'anterior pare
    for i in range (0, len(sorted_counts)):
        letter = sorted_counts[0]
        if letter[1] > valor_ant:
            esq = pare_ant
            dr = Node(letter[0], letter[1])
            node_list.append(dr)
        else:
            esq = Node(letter[0], letter[1])
            dr = pare_ant
            node_list.append(esq)
        valor_ant += letter[1] 
        pare = Node(valor_ant, -1, esq, dr)

        sorted_counts.pop(0)        
        node_list.append(pare)
        pare_ant = pare

    # Una vegada creat l'arbre fem el diccionari cridant els dos primer fills
    create_dict(node_list, node_list[-1].left, 0, node_list[-1].code)
    create_dict(node_list, node_list[-1].right, 1, node_list[-1].code)

    for i in node_list:
        if i.value != -1:
            codes[i.node] = i.code
    
    return codes

In [20]:
def encode(text, diccionari):
    """
    Donat un text a codificar i un diccionari de conversió, codifica el text.
    
    Params
    ======
    :text: El text que volem codificar
    :diccionari: El diccionari de conversió que farem servir
    
    Returns
    =======
    :code: Una representació del text usant només els caràcters '.' i '-'
    """
    code = ""

    for letter in text:
        code += diccionari[letter]
    
    return code

In [21]:
def decode(text, diccionari):
    """
    Donat un text a decodificar i un diccionari de conversió, decodifica el text.
    
    Params
    ======
    :text: El text que volem decodificar (caràcters '.' i '-')
    :diccionari: El diccionari de conversió que hem fet servir per codificar
    
    Returns
    =======
    :code: El text resultant de la decodificació.
    """
    code = ""

    caracter = ""
    # Invertim el diccionari
    diccionari = {valor: clave for clave, valor in diccionari.items()}
    n = 0

    # Anem afegint caracters fins trobar una coincidencia al diccionari
    for i in range(len(text)):
        caracter = text[n : i+1]
        if caracter in diccionari.keys():
            code += diccionari[caracter]
            n = i + 1

    return code

In [22]:
def alphabet(text):
    """
    Funció per comprovar que el codi s'executa correctament. No modifiqueu aquesta funció.
    
    Params
    ======
    :text: Missatge que volem fer servir per provar les funcions.
    """
    
    print("Text a codificar:", text)
    
    counts = compute_frequency(text)
    print("Diccionari de freqüències:", counts)
    
    codes = assign_codes(text, counts)
    print("Diccionari de conversió:", codes)
    
    codi = encode(text, codes)
    print("Codificat:", codi)
    print("Decodificat:", decode(codi, codes))

In [ ]:
# text_a_codificar = "ABBCACCCDBAAABCAACCCB"
text_a_codificar = "ABABAC"
alphabet(text_a_codificar)

In [ ]:
text = "ABBCACCCDBAAABCAAC.d,flkajefkjanvkjbamefaCCB.sadsajdha"
alphabet(text)